## Dataset: <a href="https://www.kaggle.com/plameneduardo/sarscov2-ctscan-dataset"> SARS-COV-2 Ct-Scan Dataset </a>

## Requisitos

In [ ]:
pip install Keras

In [ ]:
pip install tensorflow

## Importações

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize

## Carregamento e redimensionamento das imagens

In [ ]:
datadir = "C:\\Users\\robso\\CNN COVID-19\\"
categories = ['NON-COVID', 'COVID']
x = []
y = []

for category in categories:
    path = os.path.join(datadir, category)
    for image in os.listdir(path):
        images = cv2.imread(os.path.join(path, image), cv2.IMREAD_GRAYSCALE)
        images = resize(images, (256, 256, 3), mode = 'constant', preserve_range = True)
        x.append(images)
        y.append(categories.index(category))

In [ ]:
noncovid = 0
covid = 0

for i in range(len(x)):
    if y[i] == 0:
        noncovid = noncovid + 1
    elif y[i] == 1:
        covid = covid + 1
        
print("Número de imagens: ", len(x))
print("Número de imagens de COVID-19 negativo: ", noncovid)
print("Número de imagens de COVID-19 positivo: ", covid)

In [ ]:
y = np.array(y)
X = np.zeros(((len(x), 256, 256, 3)), dtype='uint8')

for i in range(len(x)):
    X[i,:,:]=x[i]/255

## Importações necessárias para a construção da CNN e disponibilização dos resultados

In [ ]:
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns

## Separação entre treino e teste

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
print("Imagens de treino: ", len(x_train))
print("Imagens de teste: ", len(x_test))

## Construção do modelo

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dropout(0.7))
model.add(Dense(128, activation='relu'))

model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

model.summary()

## Treinamento da rede

In [ ]:
results = model.fit(x_train, y_train, epochs=30, validation_data=(x_test,y_test), batch_size=8)

## Gráficos de acurácia e perda

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(121)
plt.title('Acurácia')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.legend(['Treino', 'Validação'], loc='upper left')

plt.subplot(122)
plt.title('Perda')
plt.ylabel('Perda')
plt.xlabel('Época')
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.legend(['Treino', 'Validação'], loc='upper left')

plt.show()

## Teste da rede

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.round(y_pred)

cmat=confusion_matrix(y_test,y_pred,labels=[0,1])
cm_df = pd.DataFrame(cmat)

tp = cmat[1][1]
fp = cmat[0][1]
tn = cmat[0][0]
fn = cmat[1][0]

accuracy = ((tp+tn)*100)/np.sum(cmat)
precision = (tp*100)/(tp+fp)
sensibility = (tp*100/(tp+fn))
specificity = (tn*100)/(fp+tn)

print('Métrica de avaliação\t\tResultado')
print('---------------------------------------------')
print('Acurácia:\t\t',accuracy,"%")
print('Precisão:\t\t',precision,"%")

print('Sensibilidade:\t\t',sensibility,"%")
print('Especificidade:\t\t',specificity,"%")
                   
plt.title('Teste')
sns.heatmap(cm_df, annot=True, fmt="d", cmap=plt.cm.Oranges)
plt.show()